In [155]:
from datasets.load import dataset_module_factory

from docuverse.utils import open_stream
import os
import json
import pandas as pd
import random
from docuverse.utils import get_param
from tqdm._tqdm_notebook import tqdm
import csv


In [153]:
datadir="../benchmark/ibm_search"
query_file=os.path.join(datadir,"queries.jsonl")
dev_file=os.path.join(datadir,"qrels/dev.tsv")
corpus=os.path.join(datadir, "corpus.jsonl.bz2")

In [189]:
queries=pd.read_json(open(query_file), lines=True)
qtext2id = {}
for i, q in enumerate(queries['_id']):
    s = queries['text'][i].replace("\\\\n", " ").replace("\\n", " ").replace("\n", " ")
    
    qtext2id[s] = q
dev=pd.read_csv(dev_file, delimiter="\t")

In [174]:
qtext2id["i want to register myself for ibm service link to check the ptf's and apars"]

'test95'

In [157]:

docs = []
id2pos = {}
url2id = {}
def read_id2url(file):
    vals = {}
    with open_stream(file) as inp:
        for i, line in tqdm(enumerate(inp), total=1710114):
            d = json.loads(line)
            d['_id'] = d['_id'].split("-")[0]
            id2pos[int(d['_id'])] = i
            url2id[get_param(d, "metadata.url")] = int(d['_id'])
            docs.append(d)
            vals[i] = {'url': d['metadata']['url'], 'dcc': d['metadata']['dcc']}
            
    return vals

vals = read_id2url(corpus)

  0%|          | 0/1710114 [00:00<?, ?it/s]

In [165]:
test_documents = []
with open(os.path.join(datadir, "dev.csv")) as inp:
    r = csv.reader(inp, delimiter=",")
    for line in r:
        if line[0]=='query':
            continue
        urls = line[1].split(",")
        for url in urls:
            if url not in url2id:
                print(f"The url {url} is not in the corpus!")
            upos = url2id[url]
            if upos not in test_documents:
                test_documents.append(upos)

In [169]:
with open(os.path.join(datadir, "dev.urls")) as inp:
    for line in inp:
        url = line.strip()[1:-1]
        if url not in url2id:
            print(f"The url {url} is not in the corpus!")
        upos = url2id[url]
        if upos not in test_documents:
            test_documents.append(upos)

In [191]:
with open(os.path.join(datadir, "qrels/dev.fixed.csv"), "w") as out, open(os.path.join(datadir, "dev.csv")) as inp:
    w = csv.writer(out, delimiter="\t")
    w.writerow(["query-id", "corpus-id", "score"])
    r = csv.reader(inp, delimiter=",")
    for line in r:
        if line[0]=='query':
            continue
        urls = line[1].split(",")
        line[0] = line[0].replace("\\n", " ")
        if line[0] not in qtext2id:
            print(f"The query \"{line[0]}\" is not in the query file!")
        else:
            qid = qtext2id[line[0]]
            for url in urls:
                if url not in url2id:
                    print(f"The url {url} is not in the corpus!")
                upos = url2id[url]
                w.writerow([qid, upos, 1])
                # print([qid, upos, 1])

In [131]:
id2query = {q: i for i, q in enumerate(queries['_id'])}

In [188]:
[(k,v) for k, v in qtext2id.items() if k.find("this system is currently in a hyperswap topology")>=0]

[("this system is currently in a hyperswap topology.  we need to move one iogrp (982) to a new data center.  i requested a procedure to complete the conversion from hs to standard.  below is the procedure i was given.   i need to confirm step 2.  should this be completed last  will steps 3, 4, 5 be impacted by the topology change  please confirm.  thank you!\\n\\n### from previous case opened a few months ago ###\\n\\nhere's the procedure for de-configuring hyperswap\\n\\nthis process is non-disruptive, as long as hosts and san are properly configured to allow access to vdisks at the site with the surviving volume copy.\\n\\nthe general procedure is as follows:\\n\\n 1. delete all active-active rcrelationships. do this by running 'rmvolumecopy -site x -removefcmaps <volume_name>', where x is the site whose copy you want to discard. this will automatically delete the active-active rcrelationship and change\\nvolumes.\\n\\ndo not use rmrcrelationship, as this will not change the host map

In [182]:
qtext2id['how do i upgrade \\nfrom one storage scale version to another']

KeyError: 'how do i upgrade \\nfrom one storage scale version to another'

In [132]:
test_documents = []
for i, q in enumerate(dev['query-id']):
    if dev['corpus-id'][i] not in test_documents:
        test_documents.append(dev['corpus-id'][i])

98

In [171]:

print(f"There are {len(test_documents)} test documents.")
while True:
    j = random.randint(0, len(vals)-1)
    j_id = int(docs[j]['_id'])
    if j_id in test_documents:
        continue
    test_documents.append(j_id)
    if len(test_documents) == 10000:
        break
print(f"There are {len(test_documents)} selected documents.")

docmap = {}
with open(os.path.join(datadir, "corpus.10k.dev.jsonl"), "w") as out:
    for j, i in enumerate(test_documents): # enumerate(sorted(test_documents)):
        print(json.dumps(docs[id2pos[i]]), file=out)


There are 9734 test documents.
There are 10000 selected documents.


In [150]:
docs[1502628]

{'title': 'Spectrum Virtualize Remote Support IP addresses are changing',
 '_id': '1502629',
 'text': 'News Abstract The Remote Support feature that allows IBM to remotely connect to Spectrum Virtualize products requires the device to initiate a connection back to IBM on a set of fixed IP addresses. The IP addresses used for this feature are changing in 2022. Important Update: The deadline for making this update is now earlier than previously announced. More details in the content section. Clients that use Remote Support need to update their product and any firewall, HTTP proxies, or Remote Support proxies to continue to use Remote Support without interruption. A number of other IBM Storage products are also affected by this change. Links to details about the other products are included at the end of the document. Content To continue to allow IBM to provide remote support for your Spectrum Virtualize system, you need to make some changes to your environment before the end of December 2

In [115]:
# with open(os.path.join(datadir, "qrels/dev.small.tsv"), "w") as out:
for i, _ in enumerate(dev['query-id']):
    did = dev['corpus-id'][i]
    # dev['corpus-id'][i] = docmap[did]
    dev.loc[i, 'corpus-id'] = docmap[did]


In [116]:
dev.to_csv(os.path.join(datadir, "qrels/dev.small.tsv"), header=['query-id', 'corpus-id', 'score'], sep="\t", index=False)

In [112]:
dev.keys()

Index(['query-id', 'corpus-id', 'score'], dtype='object')